# RDD

Las Resilents Data Base son datos distribuidos y son inmutables

In [27]:
import findspark
findspark.init()

In [28]:
from pyspark import SparkContext

In [29]:
sc = SparkContext.getOrCreate()

In [30]:
rdd = sc.parallelize([1,2,3])
type(rdd)

pyspark.rdd.RDD

In [31]:
rdd.collect()

[1, 2, 3]

In [32]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [33]:
!ls resources/files

deporte.csv           deportistaError.csv   modelo_relacional.jpg
deportista.csv        evento.csv            paises.csv
deportista2.csv       juegos.csv            resultados.csv


In [34]:
path = 'resources/files/'
equiposOlimpicosRDD = sc.textFile(path + "paises.csv") \
    .map(lambda line: line.split(","))

In [36]:
equiposOlimpicosRDD.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

## Siempre tenemos que ejecutar  un stop para que no se consuma la memoria.

In [26]:
sc.stop()

Casí siempre hay que ocupar tuplas en lugar de arreglos.

In [37]:
equiposOlimpicosRDD.map(lambda x : (x[2])).distinct().count()

231

# Operador GroupBy

La función mapValues nos ayuda a poder agrupar los valores de nuestra agrupación, este mapeo de valores le aplicamos una función

el elemento x[2] es la llave y el elemento x[1] son los datos.

In [77]:
equiposOlimpicosRDD.map(lambda x : (x[2], x[1])) \
    .groupByKey() \
    .mapValues(len) \
    .take(5)

[('ARG', 18), ('AFG', 1), ('IRL', 7), ('POR', 21), ('AUS', 23)]

# Operador Filter 

In [43]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l : "ARG" in l )
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

# Instrucción Collect
Este instrucción lo que hace es traernos todos los datos que estamos solicitando, por lo que solamente debe ser ejecutada cuando sepamos la longitud de los datos que vamos a consultar.


## Operador count vs counApprox
El operdador **count** lo que va haces es contar todos los registros que tenemos en nuestro RDD sin embargo si dicho registro tiene millones de registros podemos acotar la instrucción a cierto número de registros.

In [48]:
equiposOlimpicosRDD.countApprox(10)

1185

In [51]:
!ls resources/files

deporte.csv           deportistaError.csv   modelo_relacional.jpg
deportista.csv        evento.csv            paises.csv
deportista2.csv       juegos.csv            resultados.csv


In [52]:
deportistaOlimpicoRDD = sc.textFile(path + "deportista.csv") \
    .map(lambda l: l.split(", "))

deportistaOlimpicoRDD2 = sc.textFile(path + "deportista2.csv") \
    .map(lambda l: l.split(","))

Esto lo que hizo es que basicamente le pego a nuestro dataframe los datos como columnas

In [59]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD \
    .union(deportistaOlimpicoRDD2)

Para forzar a utilizar todo el contenido y ver que todo este bien ocupamos el comando count.

In [62]:
deportistaOlimpicoRDD.top(2)

[['deportista_id,nombre,genero,edad,altura,peso,equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

# Join entre la tabla deportista con la tabla juego
<img src= "resources/files/modelo_relacional.jpg" width = 450>

La unión que se va a realizar es entre la columna equipo id, para eso lo seleccionamos del último elemento de la lista.

In [66]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [65]:
deportistaOlimpicoRDD.take(2)

[['deportista_id,nombre,genero,edad,altura,peso,equipo_id'],
 ['1,A Dijiang,1,24,180,80,199']]

## Operador take y takeSample
A la hora de realizar nuestro join es posible que haya ocurrido algun error por lo que es necesario que revisemos mediante 
el operador takeSample ya que con este operador se generará una muesta aleatoria de nuestros datos, el operador take smaple lo hace recibe los siguiente:
- Si deseamos que haya valores repetidos en mi RDD.
- Cuantos valores deseo visualizar.
- Semilla aleatoria. 

In [68]:
deportistaOlimpicoRDD.map(lambda l : [l[-1],l[0:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0],x[-1]])) \
    .take(5)

[('778', (['67964', 'Ildefonso Lee Valdes', '1', '28', '165', '59'], 'PAN')),
 ('778', (['68986', 'Sal Leslie', '1', '20', '170', '62'], 'PAN')),
 ('778', (['69268', 'Yvette Lewis', '2', '31', '175', '75'], 'PAN')),
 ('778', (['73993', 'Calixto Malcon', '1', '21', '190', '86'], 'PAN')),
 ('778', (['78556', 'Ramn Mea', '1', '20', '162', '57'], 'PAN'))]

In [69]:
deportistaOlimpicoRDD.map(lambda l : [l[-1],l[0:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0],x[-1]])) \
    .takeSample(False,5, 25)

[('96', (['96916', 'Marcel Prvost', '1', '0', '0', '0'], 'BEL')),
 ('1003', (['83637', 'Ahmed Musa Jouda', '1', '27', '180', '58'], 'SUD')),
 ('413', (['119857', 'Margaret ThomasNeale', '2', '20', '152', '52'], 'GBR')),
 ('619',
  (['98446', 'Annastasia Karen Silva Raj', '2', '20', '166', '45'], 'MAS')),
 ('507', (['91948', 'Ilario Passerini', '1', '24', '174', '75'], 'ITA'))]

## Diagrama de resultados

In [73]:
resultado = sc.textFile(path+ "resultados.csv") \
    .map(lambda l : l.split(","))

In [75]:
resultadoGanador = resultado.filter(lambda l : 'NA'not in l[1])

In [76]:
resultadoGanador.take(10)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14'],
 ['42', 'Bronze', '17', '17', '21'],
 ['43', 'Gold', '17', '17', '22'],
 ['45', 'Gold', '17', '17', '24'],
 ['49', 'Gold', '17', '17', '28'],
 ['51', 'Bronze', '17', '19', '22']]

# Join entres Resultado , Deportista,equipos Olimpicos
<img src= "resources/files/modelo_relacional.jpg" width = 450>

Para realizar un join spark lo hara tomando el primer elemento como llave primaria

In [101]:
deportistaOlimpicoRDD.take(3)

[['deportista_id,nombre,genero,edad,altura,peso,equipo_id'],
 ['1,A Dijiang,1,24,180,80,199'],
 ['2,A Lamusi,1,23,170,60,199']]

In [109]:
equiposOlimpicosRDD.take(3)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX']]

In [100]:
resultadoGanador.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

- -

--

--

Código que no funciona :(


deportistas =  equiposOlimpicosRDD.map(lambda l : [l[0], l[-1]]) \
                .join(deportistaOlimpicoRDD.map(lambda l : [l[-1], l[:-1]])) \
                .map(lambda l :[l[1][1][0]])  \
                .join()
               

In [97]:
deportistaPaises = deportistaOlimpicoRDD \
    .map(lambda l : [l[-1], l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda l: [l[0], l[2]]))

[('778', (['67964', 'Ildefonso Lee Valdes', '1', '28', '165', '59'], 'PAN')),
 ('778', (['68986', 'Sal Leslie', '1', '20', '170', '62'], 'PAN'))]

# Operaciones Númericas

In [142]:
valoresMedallas = { "Gold": 7,
                    "Silver": 5,
                    "Bronze": 4}

In [152]:
paisesMedallas = deportistaPaises.join(resultadoGanador)

Se cambian los valores de nuestro dataframe

In [153]:
paisesMedallas = paisesMedallas \
    .map(lambda x:[ x[1][0][-1], valoresMedallas[x[1][1]]])

Para poder realizar oparaciones en los RDD hacemo uso de la biblioteca operator.

In [156]:
from operator import add
conclusion = paisesMedallas.reduceByKey((add)).sortBy(lambda x : x[1],ascending = False)

Condensado neto de las medallas que gano cada país en los juegos olimpicos a lo largo de la historia

In [157]:
conclusion.take(10)

[('CAN', 47901),
 ('ARG', 18198),
 ('HUN', 15672),
 ('MEX', 9288),
 ('RSA', 6144),
 ('BLR', 6120),
 ('LTU', 2775),
 ('MGL', 2085),
 ('USA', 1971),
 ('AZE', 1890)]

In [ ]:
depor